In [1]:
import cv2 
import numpy as np
import urllib
from PIL import Image
import requests
import sys

import rtde.rtde as rtde
import rtde.rtde_config as rtde_config

In [4]:
hostname = "10.1.91.167"
ROBOT_PORT = 30004
config_filename = 'control_loop_configuration.xml'
keep_running = True

In [44]:
# Bild aus Website einlesen als Video
while True:
    # download the image, convert it to a NumPy array, and then read
    resp = urllib.request.urlopen(f"http://{hostname}:4242/current.jpg?type=color")
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    img = cv2.imdecode(image, cv2.IMREAD_COLOR)
    # return the image
    cv2.imshow("Cam", img)
    if cv2.waitKey(1) == ord('q'):
        break
    
cv2.destroyAllWindows()


In [ ]:
# Bild aus Website auslesen statisches Bild
resp = urllib.request.urlopen(f"http://{hostname}:4242/current.jpg?type=color")
image = np.asarray(bytearray(resp.read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)

cv2.imshow("Cam", image)
cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
# Bild aus Website auslesen statisches Bild --- Muss noch getestet werden
im = Image.open(requests.get(f"http://{hostname}:4242/current.jpg?type=color", stream=True).raw)
im.show("Bild von Website")

In [10]:
# Konfiguationsfile für Kommunikation laden
conf = rtde_config.ConfigFile(config_filename)
input_names, input_types = conf.get_recipe('setp')
output_names, output_types = conf.get_recipe('getp')

In [11]:
# RTDE Client erstellen und verbinden
con = rtde.RTDE(hostname, ROBOT_PORT)
con.connect() 

In [30]:
# Virtuelle Maschine
# RTDE Client erstellen und verbinden
con = rtde.RTDE('10.1.180.157', ROBOT_PORT)
con.connect()

In [12]:
# setup recipes 

input_vals = con.send_input_setup(input_names, input_types)
output_vals = con.send_output_setup(output_names, output_types)

In [13]:
input_vals.__dict__

{'input_int_register_0': None, 'input_int_register_1': None, 'recipe_id': 1}

In [14]:
input_vals.input_int_register_0 = 0
input_vals.input_int_register_1 = 0

In [15]:
#start data synchronization
if not con.send_start():
    sys.exit()

In [39]:
last_output = 0
input_vals.input_int_register_0 = 0
#print(input_vals.input_int_register_0)

while keep_running:
    # receive the current state
    output_vals = con.receive()
    
    if output_vals is None:
        break

    print("Last Output: " + str(last_output))
    print("Current Output: " + str(output_vals.output_int_register_0))

    if last_output != output_vals.output_int_register_0:
        input_vals.input_int_register_0 += 1

    last_output = output_vals.output_int_register_0 

    
    print(input_vals.input_int_register_0)
    # Einfach mal senden
    con.send(input_vals)
    cv2.waitKey(1000)   
    output_vals = con.receive()

    if  output_vals.output_int_register_0 >= 20:
        break


    continue
    # do something...
    if output_vals.output_int_register_0 != 0:
        print("Hello")
        # send new setpoint        
        con.send(input_vals.__dict__["input_int_register_0"])


Last Output: 0
Current Output: 20
1
Last Output: 20
Current Output: 1
2
Last Output: 1
Current Output: 2
3
Last Output: 2
Current Output: 3
4
Last Output: 3
Current Output: 4
5
Last Output: 4
Current Output: 5
6
Last Output: 5
Current Output: 6
7
Last Output: 6
Current Output: 7
8
Last Output: 7
Current Output: 8
9
Last Output: 8
Current Output: 9
10


KeyboardInterrupt: 

In [16]:
# RTDE Client verbindung trennen
con.disconnect()